In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ner_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [3]:
EPOCHS = 100
dataset_name = 'hotel_zh'
# dataset_name = 'ccks2021'
model, (train_x, train_y), (test_x, test_y) = ner_model.create_model(dataset_name=dataset_name)

Some layers from the model checkpoint at bert-base-chinese were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-chinese.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Model: "model"
__________________________________________________________________________________________________
Layer 

In [4]:

# train model
histroy = model.fit(train_x, train_y, 
                    batch_size=16,
                    epochs=EPOCHS, 
                    validation_data=(test_x, test_y),
                    # validation_data=(dev_x, dev_y),
                    callbacks=[ModelCheckpoint('{}/crf_{}.h5'.format(ner_model.model_dir, dataset_name),
                            monitor='val_loss',
                            mode='min',
                            save_best_only=True,
                            save_weights_only=True),
                                EarlyStopping(
                    monitor='val_loss', patience=3)]
                    )
# model.save('{}/crf_{}.h5'.format(ner_model.model_dir, dataset_name))
# model.save('{}/bilstm_softmax.h5'.format(ner_model.model_dir))

Epoch 1/100


ValueError: in user code:

    /Users/derbysofti81/miniforge3/envs/transformer_no_pandas/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /Users/derbysofti81/miniforge3/envs/transformer_no_pandas/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/derbysofti81/miniforge3/envs/transformer_no_pandas/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/derbysofti81/miniforge3/envs/transformer_no_pandas/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/derbysofti81/miniforge3/envs/transformer_no_pandas/lib/python3.9/site-packages/ten

In [ ]:
histroy

In [ ]:
import matplotlib.pyplot as plt
acc = 'accuracy'
# acc = "crf_accuracy"
# val_acc = 'val_crf_accuracy'
val_acc = 'val_accuracy'
loss = "loss"
val_loss = 'val_loss'
def plot_history(history):
    plt.plot(history.history[acc])
    plt.plot(history.history[val_acc])
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel(acc)
    plt.legend([acc, val_acc], loc='lower right')
    plt.show()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['loss', 'val_loss'], loc='lower right')
    plt.show()

plot_history(history=histroy)

NameError: name 'histroy' is not defined

In [ ]:
# for y in test_y:
#     print()

In [6]:
import ner_model
# import process_data 
import hotel_process_data as process_data
# import cck_process_data as process_data
import numpy as np
# dataset_name = 'hotel_zh'
# dataset_name = 'demo'
model, (vocab, chunk_tags) = ner_model.create_model(train=False, dataset_name=dataset_name)
model.load_weights('{}/crf_{}.h5'.format(ner_model.model_dir, dataset_name))
# model.load_weights('model/bilstm_softmax.h5')

def infer(predict_text):
    # predict_text = '中华人民共和国国务院总理周恩来在外交部长陈毅和王东的陪同下，连续访问了埃塞俄比亚等非洲10国以及阿尔巴尼亚'
    str, length = process_data.process_data(predict_text, vocab)
    # print(str, length)

    raw = model.predict(str)[0][-length:]
    # print(raw)
    # result = [np.argmax(row) for row in raw]
    result = raw
    # print(result_tags)
    result_tags = [chunk_tags[i] for i in result]

    per, loc, org = '', '', ''
    brd = ''

    for s, t in zip(predict_text, result_tags):
        if t in ('B-PER', 'I-PER'):
            per += ' ' + s if (t == 'B-PER') else s
        if t in ('B-ORG', 'I-ORG'):
            org += ' ' + s if (t == 'B-ORG') else s
        if t in ('B-LOC', 'I-LOC'):
            loc += ' ' + s if (t == 'B-LOC') else s
        if t in ('B-BRD', 'I-BRD'):
            brd += ' ' + s if (t == 'B-BRD') else s

    # print(['person:' + per, 'location:' + loc, 'organzation:' + org])
    # print('location:' + loc)
    print("input:{} ==> brand:{}".format(predict_text, brd))
    return brd

cases = ['如家酒店(杭州四季青服装市场清江路店)','清沐酒店(杭州四季青服装市场清江路店)', '澳门新葡京酒店', '澳门葡京酒店', '东莞长安海悦花园大酒店', '桔子水晶深圳宝安机场碧海湾酒店', 
        '博鳌和悦景澜海景度假酒店','上海虹桥国家会展中心希尔顿欢朋酒店', '厦门磐基希尔顿酒店','徐州复兴苏宁广场苏宁雅悦酒店','古北水镇新青年艺术酒店','古北水镇古北之光温泉酒店']
for case in cases:
    infer(case)

input:如家酒店(杭州四季青服装市场清江路店) ==> brand: 如家酒店
input:清沐酒店(杭州四季青服装市场清江路店) ==> brand: 清沐酒店
input:澳门新葡京酒店 ==> brand: 京酒店
input:澳门葡京酒店 ==> brand: 葡京酒店
input:东莞长安海悦花园大酒店 ==> brand: 大酒店
input:桔子水晶深圳宝安机场碧海湾酒店 ==> brand: 桔子水晶
input:博鳌和悦景澜海景度假酒店 ==> brand: 海景度假酒店
input:上海虹桥国家会展中心希尔顿欢朋酒店 ==> brand: 欢朋酒店
input:厦门磐基希尔顿酒店 ==> brand: 希尔顿酒店
input:徐州复兴苏宁广场苏宁雅悦酒店 ==> brand: 宁雅悦酒店
input:古北水镇新青年艺术酒店 ==> brand: 古北水镇
input:古北水镇古北之光温泉酒店 ==> brand: 古北水


In [94]:
'新' in vocab
# len(vocab)

True

In [105]:
import pickle  
f=open('./data/check_data.pkl','rb')  
aa = pickle.load(f)
supplier_hotel_names, distributor_hotel_names = aa

In [109]:
for i in range(len(supplier_hotel_names)):
    supplier_hotel_name = supplier_hotel_names[i]
    distributor_hotel_name = distributor_hotel_names[i]
    supplier_infer = infer(supplier_hotel_name)
    distributor_infer = infer(distributor_hotel_name)
    if supplier_infer != distributor_infer:
        print(supplier_hotel_name, distributor_hotel_name, supplier_infer, distributor_infer)

如家·neo北京大红门地铁站店 如家酒店(北京大红门地铁站店)   如家酒店
和颐至尚酒店(北京石景山万达广场店) 和颐酒店(北京石景山万达广场店)  和颐至尚酒店  和颐酒店
和颐酒店(北京三里屯使馆区店) 和颐至尚酒店(北京三里屯使馆区店)  和颐酒店  和颐至尚酒店
和颐酒店(广州盘福路越秀公园店) 和颐至尚酒店（广州盘福路越秀公园店）  和颐酒店  和颐至尚酒店
如家酒店·neo(上海徐家汇宛平南路店) 如家·neo(上海徐家汇宛平南路店)  如家酒店 
如家酒店·neo(上海南京路步行街黄河路店) 如家·neo(上海南京路步行街黄河路店)  如家酒店 
和颐酒店(上海新国际博览中心世博园店) 和颐至尚酒店（上海新国际博览中心世博园店）  和颐酒店  和颐至尚酒店
如家酒店·neo(上海徐家汇中山西路店) 如家·neo(上海徐家汇中山西路店)  如家酒店 
和颐酒店(上海新国际博览中心芳甸路地铁站店) 和颐至尚酒店(上海新国际博览中心芳甸路地铁站店)  和颐酒店  和颐至尚酒店
和颐至尊酒店(上海南京西路地铁站店) 和颐酒店(上海南京西路领事馆店)  和颐至尊酒店  和颐酒店
和颐至格酒店(上海北京西路人民广场店) 和颐酒店(上海北京西路人民广场店)  和颐至格酒店  和颐酒店
如家酒店·neo(天津红旗路宜宾道地铁站店) 如家·neo-天津红旗路宜宾道地铁站店  如家酒店 
如家酒店·neo(重庆解放碑洪崖洞店) 如家酒店(重庆解放碑洪崖洞店)  如家酒店 解放  如家酒店
如家酒店·neo(南京新街口大行宫地铁站文昌巷店) 如家·neo(南京新街口大行宫地铁站文昌巷店)  如家酒店 新街口  新街口大
如家酒店·neo(南京夫子庙集庆门店) 如家·neo(南京夫子庙集庆门店)  如家酒店  夫子
如家酒店·neo(南京中山东路明故宫瑞金店) 如家·neo(南京中山东路明故宫瑞金店)  如家酒店  中山
如家酒店·neo(南京夫子庙地铁站平江府路店) 如家·neo(南京夫子庙地铁站平江府路店)  如家酒店  夫子
如家酒店·neo(南京九龙湖地铁站双龙大道店) 如家·neo(南京九龙湖地铁站双龙大道店)  如家酒店 
如家酒店·neo(南京桥北弘阳广场泰山新村地铁站店) 如家·neo(南京桥北弘阳广场泰山新村地铁站店)  如家酒店 
和颐酒店(成都武侯祠省

In [2]:
###
import requests
from utils.constants import HOST_IP
def tag_service(line):
    resp = requests.get("http://localhost:8082/cck_ner?text={}".format(line))
    return resp.json()['res']

import pandas as pd
import pymongo
import numpy as np
from utils.constants import HOST_IP
dataset_name = 'hotel_zh_gaodeng'
client = pymongo.MongoClient(HOST_IP, 27017)

db  = client[dataset_name]
col = db["yws_hotel_mapping"]
tag_col = db["yws_hotel_mapping_tag"]
# df = pd.DataFrame(list(col.find())) # 

# all_train_df['inter'] = all_train_df.apply(lambda row: inter(row['left_text'],row["right_text"]),axis=1)
cursor = col.find(no_cursor_timeout=True)
collection_data = [document for document in cursor]
for item in collection_data:
    # print(item)
    res = tag_service(item['hotel_name']+","+item["address"])
    item['ner_words'] = res[0]
    item['ner_tags'] = res[1]
    tag_col.insert(item)

<ipython-input-2-60e4040de50f>:28: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  tag_col.insert(item)


In [ ]:
# update tag 
def update_tag(tags, words):
    pass